<a href="https://colab.research.google.com/github/kissflow/prompt2finetune/blob/main/Prompting_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [14]:
import os
from openai import OpenAI
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')


# Make sure you have set the OPENAI_API_KEY environment variable
# For Colab, you can use the "🔑" icon on the left panel to add your API key as a secret.
# Name the secret OPENAI_API_KEY.
# If you are running this outside of Colab, you can set it as an environment variable
# in your terminal: export OPENAI_API_KEY='your-api-key'
# Or you can uncomment the line below and replace 'your-api-key' with your actual key:
# os.environ['OPENAI_API_KEY'] = 'your-api-key'

client = OpenAI(
    # This is the default and can be omitted
    api_key= userdata.get('OPENAI_API_KEY'),
)

model = "gpt-4.1-mini-2025-04-14" #"gpt-3.5-turbo"

def generate_response(prompt, model=model, max_tokens=150):
    """
    Generates a response from the OpenAI API.

    Args:
        prompt: The input prompt for the model.
        model: The OpenAI model to use (default: "gpt-3.5-turbo").
        max_tokens: The maximum number of tokens in the generated response.

    Returns:
        The text of the generated response.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=0.7, # You can adjust the temperature
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generating response: {e}"

## Zero-Shot Prompting

In [15]:
# Real-world example: Summarize a product review with zero-shot prompting.
zero_shot_prompt = """
Summarize the following product review:

Review: This vacuum cleaner is amazing! It's lightweight, powerful, and the battery lasts a long time. I was able to clean my entire house on a single charge. The attachments are also very useful for getting into tight spaces. Highly recommended!
"""

zero_shot_response = generate_response(zero_shot_prompt)
print("Zero-Shot Response:")
print(zero_shot_response)

Zero-Shot Response:
The vacuum cleaner is lightweight, powerful, and has a long-lasting battery that allows cleaning the entire house on one charge. Its attachments are effective for tight spaces. Overall, it is highly recommended.


## Few-Shot Prompting

In [16]:
# Real-world example: Summarize a product review with few-shot examples.
few_shot_prompt = """
Summarize the following product reviews:

Review: This phone is terrible. The battery dies in an hour and the camera is blurry.
Summary: Poor battery life and camera quality.

Review: I love this laptop! It's fast, lightweight, and the screen is stunning.
Summary: Fast, lightweight, and excellent display.

Review: This vacuum cleaner is amazing! It's lightweight, powerful, and the battery lasts a long time. I was able to clean my entire house on a single charge. The attachments are also very useful for getting into tight spaces. Highly recommended!
Summary:
"""

few_shot_response = generate_response(few_shot_prompt)
print("Few-Shot Response:")
print(few_shot_response)

Few-Shot Response:
Lightweight, powerful vacuum with long battery life and useful attachments; highly recommended.


## Self Reflexion

In [17]:
# Real-world example: Summarize a product review using Reflexion.

# Initial prompt
initial_review_prompt = "Summarize the following product review:\n\nThis vacuum cleaner is amazing! It's lightweight, powerful, and the battery lasts a long time. I was able to clean my entire house on a single charge. The attachments are also very useful for getting into tight spaces. Highly recommended!"

initial_review_response = generate_response(initial_review_prompt)
print("Initial Reflexion Response:")
print(initial_review_response)

# Reflection prompt to improve the summary
reflection_review_prompt = f"""
Based on the following product review and the initial summary:

Review: This vacuum cleaner is amazing! It's lightweight, powerful, and the battery lasts a long time. I was able to clean my entire house on a single charge. The attachments are also very useful for getting into tight spaces. Highly recommended!

Initial Summary: {initial_review_response}

Critique the initial summary. Does it capture all the key positive aspects mentioned in the review (lightweight, powerful, long battery life, useful attachments, highly recommended)? Suggest improvements to make the summary more comprehensive and accurate.
"""

reflection_on_review = generate_response(reflection_review_prompt)
print("\nReflection on Initial Response:")
print(reflection_on_review)

# Refined prompt incorporating the reflection
refined_review_prompt = f"""
Based on the following product review and the critique of the initial summary, provide a refined and comprehensive summary:

Review: This vacuum cleaner is amazing! It's lightweight, powerful, and the battery lasts a long time. I was able to clean my entire house on a single charge. The attachments are also very useful for getting into tight spaces. Highly recommended!

Critique: {reflection_on_review}

Refined Summary:
"""

refined_review_response = generate_response(refined_review_prompt)
print("\nRefined Reflexion Response:")
print(refined_review_response)

Initial Reflexion Response:
The vacuum cleaner is lightweight, powerful, and has a long-lasting battery that can clean an entire house on one charge. Its useful attachments help reach tight spaces. Highly recommended.

Reflection on Initial Response:
The initial summary does a good job capturing the main positive aspects mentioned in the review: the vacuum cleaner being lightweight, powerful, having a long-lasting battery capable of cleaning an entire house on one charge, and the usefulness of the attachments for tight spaces. It also includes the strong recommendation from the reviewer.

However, the summary could be improved by incorporating the sense of enthusiasm and specific praise about the battery life ("I was able to clean my entire house on a single charge") and explicitly mentioning that the vacuum is easy to use due to its lightweight design. Additionally, mentioning the variety or versatility of attachments could add clarity.

Suggested improved summary:

"This vacuum clean

## Chain of Thought Prompting

In [18]:
# Real-world example: Summarize a product review using Chain of Thought.

chain_of_thought_prompt = """
Summarize the following product review, thinking step by step:

Review: This vacuum cleaner is amazing! It's lightweight, powerful, and the battery lasts a long time. I was able to clean my entire house on a single charge. The attachments are also very useful for getting into tight spaces. Highly recommended!

Let's think step by step to summarize this review.
1. Identify the product being reviewed.
2. List the positive and negative attributes mentioned about the product.
3. Combine these attributes into a concise summary.

Note: Write down your reasoning for each step inside <thought-process> Thougts </thought-process> tag.

Summary:
"""

chain_of_thought_response = generate_response(chain_of_thought_prompt, max_tokens=300) # Increased max_tokens for detailed steps
print("Chain of Thought Response:")
print(chain_of_thought_response)

Chain of Thought Response:
<thought-process> 
1. The product being reviewed is a vacuum cleaner. 
2. Positive attributes mentioned are: lightweight, powerful, long-lasting battery (able to clean entire house on one charge), and useful attachments for tight spaces. There are no negative attributes mentioned. 
3. Combining these points, the summary should highlight the vacuum cleaner's lightweight design, strong performance, excellent battery life, and versatile attachments, concluding with a positive recommendation. 
</thought-process>

Summary: The vacuum cleaner is lightweight, powerful, has a long-lasting battery that can clean an entire house on one charge, and comes with useful attachments for tight spaces. It is highly recommended.


## Tree of Thoughts Prompting

In [19]:
# Real-world example: Summarize a product review using a simulated Tree of Thoughts.
# Note: A true Tree of Thoughts implementation is more complex and would involve
# exploring multiple reasoning paths and evaluating them. This is a simplified simulation.

def tree_of_thoughts_prompting(initial_thought_prompt, expansion_prompt, evaluation_prompt, steps=2):
    """
    Simulates a basic Tree of Thoughts approach.

    Args:
        initial_thought_prompt: The initial prompt to generate a starting thought.
        expansion_prompt: Prompt to generate subsequent thoughts based on previous ones.
        evaluation_prompt: Prompt to evaluate the generated thoughts.
        steps: Number of expansion steps to simulate.

    Returns:
        The evaluated best thought or a summary of the process.
    """
    try:
        thoughts = [generate_response(initial_thought_prompt)]
        print("Initial Thought:", thoughts[0])

        for step in range(steps):
            new_thoughts = []
            for thought in thoughts:
                expansion_response = generate_response(f"{expansion_prompt}\n\nPrevious thought: {thought}")
                # Simple split to simulate multiple thoughts
                expanded_thoughts = [t.strip() for t in expansion_response.split('\n') if t.strip()]
                new_thoughts.extend(expanded_thoughts)
            thoughts = new_thoughts
            print(f"\nThoughts after step {step + 1}:")
            for i, thought in enumerate(thoughts):
                print(f"  Thought {i+1}: {thought}")

        # Simple evaluation: Ask the model to pick the best thought or summarize
        evaluation_response = generate_response(f"{evaluation_prompt}\n\nThoughts to evaluate:\n" + "\n".join([f"- {t}" for t in thoughts]))

        return evaluation_response

    except Exception as e:
        return f"An error occurred: {e}"

# Example usage with the product review summarization
initial_thought_prompt = "Generate an initial thought about summarizing a product review."
expansion_prompt = "Expand on the following thought to generate alternative approaches for summarizing a product review."
evaluation_prompt = "Review the following thoughts for summarizing a product review and provide the best summary based on the original review: This vacuum cleaner is amazing! It's lightweight, powerful, and the battery lasts a long time. I was able to clean my entire house on a single charge. The attachments are also very useful for getting into tight spaces. Highly recommended!"


tree_of_thoughts_response = tree_of_thoughts_prompting(initial_thought_prompt, expansion_prompt, evaluation_prompt, steps=2)
print("\nTree of Thoughts Simulation Result:")
print(tree_of_thoughts_response)

Initial Thought: When summarizing a product review, it’s important to capture the overall sentiment—whether positive, negative, or mixed—while highlighting key points such as the product’s main features, performance, usability, and any common praises or complaints mentioned by the reviewer. This approach ensures the summary provides a clear and balanced snapshot for potential buyers.

Thoughts after step 1:
  Thought 1: Certainly! Expanding on the initial thought, here are several alternative approaches for summarizing a product review, each emphasizing different aspects or techniques to provide diverse perspectives and utility for potential buyers:
  Thought 2: 1. **Feature-Centric Summary**
  Thought 3: Focus primarily on the specific features of the product mentioned in the review, detailing how each feature performed or was perceived. This approach is useful when potential buyers are interested in particular functionalities rather than an overall sentiment.
  Thought 4: *Example:* 

## Meta Prompting

In [13]:
# Real-world example: Summarize a product review using Meta Prompting.

meta_prompt = """
You are an expert in summarizing product reviews. Your goal is to provide a concise and informative summary of the given review, focusing only on the key positive and negative aspects mentioned.

Here is the product review:
Review: This vacuum cleaner is amazing! It's lightweight, powerful, and the battery lasts a long time. I was able to clean my entire house on a single charge. The attachments are also very useful for getting into tight spaces. Highly recommended!

Provide the summary in the following format:
Summary: [Your concise summary here]
"""

meta_prompt_response = generate_response(meta_prompt)
print("Meta Prompting Response:")
print(meta_prompt_response)

Meta Prompting Response:
Summary: The vacuum cleaner is lightweight, powerful, with a long-lasting battery. Attachments are useful for tight spaces. Highly recommended.
